In [1]:
import z3
import json
import pprint
import numpy as np
import pandas as pd

with open ('simple.json') as file:
    data = json.load(file)

#------------SETUP-------------------------------

# List of Resource Types
resourceTypes = set([j for i in data for j in data[i]['resources'] ])

# Initialize data structures
nodeList = list()
appList = list()
nlappList = list() # List of apps that are dependencies
prAppList = list() # List of primary apps that have dependencies
a2a = []
n2n = []

# Node and App lists
for i in data:
    if data[i]['type']=='node':
        nodeList.append(i)
    if data[i]['type']=='app':
        appList.append(i)
        if data[i]['dependencies']['nonlocal']:
            prAppList.append(i)
            for dep in data[i]['dependencies']['nonlocal']:
                nlappList.append(dep)
print("Dependency apps: %s" %(nlappList))
print("Primary apps: %s" %(prAppList))
# app to app dependency matrix            
for app in appList:
    nl_deps = data[app]['dependencies']['nonlocal']
    deps = []
    for dep in appList:
        if dep in nl_deps:
            deps.append(1)
        else: 
            deps.append(0)
    a2a.append(deps)
a2a_df = pd.DataFrame(a2a, index=appList, columns=appList)        

# node to node latency matrix
n2n = []
for node in nodeList:
    id = data[node]['id']
    temp = []    
    for n2 in nodeList:
        id2 = data[n2]['id']
        lat = data[node]['latencies']['n%s%s' %(id, id2)]
        temp.append(lat)
    n2n.append(temp)
n2n_df = pd.DataFrame(n2n, index=nodeList, columns=nodeList)  

# node resource matrix
rpn= [[data[j]['resources'][k] for j in nodeList] for k in resourceTypes]
rpn_df = pd.DataFrame(rpn, index=resourceTypes, columns=nodeList)     
    
# app resource matrix
rpa= [[data[i]['resources'][k] for i in appList] for k in resourceTypes]
rpa_df = pd.DataFrame(rpa, index=resourceTypes, columns=appList)     

# app to node placment matrix
a2n = [ [ z3.Int("a%s%s" % (i+1, j+1)) for j in range(len(nodeList)) ]
            for i in range(len(appList))]
a2n_df = pd.DataFrame(a2n, index=appList, columns=nodeList) 

print("n2n: \n%s" %n2n_df)
print("a2a: \n%s" %a2a_df)
print("rpn: \n%s" %rpn_df)
print("rpa: \n%s" %rpa_df)

print("a2n: \n%s" %a2n_df)


Dependency apps: ['APP2']
Primary apps: ['APP1']
n2n: 
       FSSN2  FSSN1  FSSN3  FSSN4  FSSN5
FSSN2      0      1      1      1      1
FSSN1      1      0      1      1      1
FSSN3      1      1      0      1      1
FSSN4      1      1      1      0      1
FSSN5      1      1      1      1      0
a2a: 
      APP2  APP1
APP2     0     0
APP1     1     0
rpn: 
        FSSN2  FSSN1  FSSN3  FSSN4  FSSN5
memory    512   1024    512    512    512
rpa: 
        APP2  APP1
memory   128   256
a2n: 
     FSSN2 FSSN1 FSSN3 FSSN4 FSSN5
APP2   a11   a12   a13   a14   a15
APP1   a21   a22   a23   a24   a25


In [2]:
a2n_df

,FSSN2,FSSN1,FSSN3,FSSN4,FSSN5
APP2,a11,a12,a13,a14,a15
APP1,a21,a22,a23,a24,a25


In [3]:
n2n_df

,FSSN2,FSSN1,FSSN3,FSSN4,FSSN5
FSSN2,0,1,1,1,1
FSSN1,1,0,1,1,1
FSSN3,1,1,0,1,1
FSSN4,1,1,1,0,1
FSSN5,1,1,1,1,0


In [4]:
a2a_df

,APP2,APP1
APP2,0,0
APP1,1,0


In [5]:
rpn_df

,FSSN2,FSSN1,FSSN3,FSSN4,FSSN5
memory,512,1024,512,512,512


In [6]:
rpa_df

,APP2,APP1
memory,128,256


In [7]:
#------------Constraints-------------------------------
# variables in a2n matrix must be binary

a2n_domain = [z3.Or(a2n_df.loc[app, node]==0, a2n_df.loc[app, node]==1) \
 for app in nlappList \
 for node in nodeList]

# An intial deployment of apps that have dependencies
a2n_df.loc['APP1'] = 1
#a2n_df.loc['APP1', 'FSSN2'] = 1
#a2n_df.loc['APP1', 'FSSN3'] = 1
print (a2n_df)
a2n_domain


     FSSN2 FSSN1 FSSN3 FSSN4 FSSN5
APP2   a11   a12   a13   a14   a15
APP1     1     1     1     1     1


[Or(a11 == 0, a11 == 1),
 Or(a12 == 0, a12 == 1),
 Or(a13 == 0, a13 == 1),
 Or(a14 == 0, a14 == 1),
 Or(a15 == 0, a15 == 1)]

In [8]:
# Max capacity constraint
# Resources required by apps deployed on a node cannot exceed that nodes resources
app_util = rpa_df.dot(a2n_df)
print(app_util)
node_util = rpn_df - app_util
print(node_util)
rsrc_constraint = [node_util.loc[rsrc,node]>=0 \
                   for rsrc in resourceTypes \
                   for node in nodeList]
rsrc_constraint

                FSSN2          FSSN1          FSSN3          FSSN4  \
memory  128*a11 + 256  128*a12 + 256  128*a13 + 256  128*a14 + 256   

                FSSN5  
memory  128*a15 + 256  
                        FSSN2                   FSSN1                  FSSN3  \
memory  512 - (128*a11 + 256)  1024 - (128*a12 + 256)  512 - (128*a13 + 256)   

                        FSSN4                  FSSN5  
memory  512 - (128*a14 + 256)  512 - (128*a15 + 256)  


[512 - (128*a11 + 256) >= 0,
 1024 - (128*a12 + 256) >= 0,
 512 - (128*a13 + 256) >= 0,
 512 - (128*a14 + 256) >= 0,
 512 - (128*a15 + 256) >= 0]

In [9]:
# Must Deploy Dependencies constraint
# Assuming the primary app is deployed, all of it's dependencies must be deployed
mdd = [a2n_df.loc[dep].sum() > 0 for dep in nlappList]
mdd

[a11 + a12 + a13 + a14 + a15 > 0]

In [10]:
print([n2n_df.loc[src,dst] for src in nodeList for dst in nodeList])

[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0]


In [11]:
n2n_df

,FSSN2,FSSN1,FSSN3,FSSN4,FSSN5
FSSN2,0,1,1,1,1
FSSN1,1,0,1,1,1
FSSN3,1,1,0,1,1
FSSN4,1,1,1,0,1
FSSN5,1,1,1,1,0


In [12]:
# Latency 
numNodes = len(nodeList) # number of FSSNs
markings = list() # The equations for each possible placement of the dependencies

#TODO
## ITERATE THROUGH PRIMARY APPS
for srcNode in nodeList:
    print (srcNode)
    for prApp in prAppList:
        if a2n_df.loc[app, srcNode] == 1:            
            ### ITERATE THROUGH ALL DEPENDENCIES
            # iterate through the 2^numNodes potential dependency placements
            for i in range(2**numNodes):
                mark = format(i, 'b').zfill(numNodes) # express placment in binary    
                #print(mark)
                #if dependency is not placed skip.
                if mark == format(0,'b').zfill(numNodes):
                    continue

                marking = list() # A placment equation e.g (1 - a11)*(1 - a12)*a13
                latency = [] # latencies between node hosting primary app and all nodes hosting the dependency for a particular placement. 
                print ("-----NEW MARKING-----")
                # iterate through nodes using the binary placement to generate marking equation
                for i, node in enumerate(nodeList):
                    #print ("mark %s" %mark)
                    #print ("i %s " %i)
                    #print ("mark[i] %s" %type(int(mark[i]))) 
                    # add marking to list and if binary has a 1, add latency to list of possible latencies
                    if int(mark[i]) == 1:
                        marking.append(a2n_df.loc['APP2', node])
                        latency.append(n2n_df.loc[srcNode, node])
                    if int(mark[i]) == 0:
                        marking.append(1 - a2n_df.loc['APP2',node])
                print("latency %s" %latency)
                print("min latency %s" %min(latency))
                print("marking: %s" %marking)
                print("marking * latency: %s" %(z3.Product(marking) * min(latency).item()))
                markings.append(z3.Product(marking) * min(latency).item())

print(markings)
latency_obj = z3.Sum(markings)
pprint.pprint(latency_obj)

    



FSSN2
-----NEW MARKING-----
latency [1]
min latency 1
marking: [1 - a11, 1 - a12, 1 - a13, 1 - a14, a15]
marking * latency: (1 - a11)*(1 - a12)*(1 - a13)*(1 - a14)*a15*1
-----NEW MARKING-----
latency [1]
min latency 1
marking: [1 - a11, 1 - a12, 1 - a13, a14, 1 - a15]
marking * latency: (1 - a11)*(1 - a12)*(1 - a13)*a14*(1 - a15)*1
-----NEW MARKING-----
latency [1, 1]
min latency 1
marking: [1 - a11, 1 - a12, 1 - a13, a14, a15]
marking * latency: (1 - a11)*(1 - a12)*(1 - a13)*a14*a15*1
-----NEW MARKING-----
latency [1]
min latency 1
marking: [1 - a11, 1 - a12, a13, 1 - a14, 1 - a15]
marking * latency: (1 - a11)*(1 - a12)*a13*(1 - a14)*(1 - a15)*1
-----NEW MARKING-----
latency [1, 1]
min latency 1
marking: [1 - a11, 1 - a12, a13, 1 - a14, a15]
marking * latency: (1 - a11)*(1 - a12)*a13*(1 - a14)*a15*1
-----NEW MARKING-----
latency [1, 1]
min latency 1
marking: [1 - a11, 1 - a12, a13, a14, 1 - a15]
marking * latency: (1 - a11)*(1 - a12)*a13*a14*(1 - a15)*1
-----NEW MARKING-----
latency [

marking * latency: a11*a12*(1 - a13)*(1 - a14)*a15*0
-----NEW MARKING-----
latency [1, 0, 1]
min latency 0
marking: [a11, a12, 1 - a13, a14, 1 - a15]
marking * latency: a11*a12*(1 - a13)*a14*(1 - a15)*0
-----NEW MARKING-----
latency [1, 0, 1, 1]
min latency 0
marking: [a11, a12, 1 - a13, a14, a15]
marking * latency: a11*a12*(1 - a13)*a14*a15*0
-----NEW MARKING-----
latency [1, 0, 1]
min latency 0
marking: [a11, a12, a13, 1 - a14, 1 - a15]
marking * latency: a11*a12*a13*(1 - a14)*(1 - a15)*0
-----NEW MARKING-----
latency [1, 0, 1, 1]
min latency 0
marking: [a11, a12, a13, 1 - a14, a15]
marking * latency: a11*a12*a13*(1 - a14)*a15*0
-----NEW MARKING-----
latency [1, 0, 1, 1]
min latency 0
marking: [a11, a12, a13, a14, 1 - a15]
marking * latency: a11*a12*a13*a14*(1 - a15)*0
-----NEW MARKING-----
latency [1, 0, 1, 1, 1]
min latency 0
marking: [a11, a12, a13, a14, a15]
marking * latency: a11*a12*a13*a14*a15*0
FSSN3
-----NEW MARKING-----
latency [1]
min latency 1
marking: [1 - a11, 1 - a12, 

marking: [1 - a11, 1 - a12, 1 - a13, 1 - a14, a15]
marking * latency: (1 - a11)*(1 - a12)*(1 - a13)*(1 - a14)*a15*0
-----NEW MARKING-----
latency [1]
min latency 1
marking: [1 - a11, 1 - a12, 1 - a13, a14, 1 - a15]
marking * latency: (1 - a11)*(1 - a12)*(1 - a13)*a14*(1 - a15)*1
-----NEW MARKING-----
latency [1, 0]
min latency 0
marking: [1 - a11, 1 - a12, 1 - a13, a14, a15]
marking * latency: (1 - a11)*(1 - a12)*(1 - a13)*a14*a15*0
-----NEW MARKING-----
latency [1]
min latency 1
marking: [1 - a11, 1 - a12, a13, 1 - a14, 1 - a15]
marking * latency: (1 - a11)*(1 - a12)*a13*(1 - a14)*(1 - a15)*1
-----NEW MARKING-----
latency [1, 0]
min latency 0
marking: [1 - a11, 1 - a12, a13, 1 - a14, a15]
marking * latency: (1 - a11)*(1 - a12)*a13*(1 - a14)*a15*0
-----NEW MARKING-----
latency [1, 1]
min latency 1
marking: [1 - a11, 1 - a12, a13, a14, 1 - a15]
marking * latency: (1 - a11)*(1 - a12)*a13*a14*(1 - a15)*1
-----NEW MARKING-----
latency [1, 1, 0]
min latency 0
marking: [1 - a11, 1 - a12, a13

(1 - a11)*(1 - a12)*(1 - a13)*(1 - a14)*a15*1 +
(1 - a11)*(1 - a12)*(1 - a13)*a14*(1 - a15)*1 +
(1 - a11)*(1 - a12)*(1 - a13)*a14*a15*1 +
(1 - a11)*(1 - a12)*a13*(1 - a14)*(1 - a15)*1 +
(1 - a11)*(1 - a12)*a13*(1 - a14)*a15*1 +
(1 - a11)*(1 - a12)*a13*a14*(1 - a15)*1 +
(1 - a11)*(1 - a12)*a13*a14*a15*1 +
(1 - a11)*a12*(1 - a13)*(1 - a14)*(1 - a15)*1 +
(1 - a11)*a12*(1 - a13)*(1 - a14)*a15*1 +
(1 - a11)*a12*(1 - a13)*a14*(1 - a15)*1 +
(1 - a11)*a12*(1 - a13)*a14*a15*1 +
(1 - a11)*a12*a13*(1 - a14)*(1 - a15)*1 +
(1 - a11)*a12*a13*(1 - a14)*a15*1 +
(1 - a11)*a12*a13*a14*(1 - a15)*1 +
(1 - a11)*a12*a13*a14*a15*1 +
a11*(1 - a12)*(1 - a13)*(1 - a14)*(1 - a15)*0 +
a11*(1 - a12)*(1 - a13)*(1 - a14)*a15*0 +
a11*(1 - a12)*(1 - a13)*a14*(1 - a15)*0 +
a11*(1 - a12)*(1 - a13)*a14*a15*0 +
a11*(1 - a12)*a13*(1 - a14)*(1 - a15)*0 +
a11*(1 - a12)*a13*(1 - a14)*a15*0 +
a11*(1 - a12)*a13*a14*(1 - a15)*0 +
a11*(1 - a12)*a13*a14*a15*0 +
a11*a12*(1 - a13)*(1 - a14)*(1 - a15)*0 +
a11*a12*(1 - a13)*(1 - a14)*

In [13]:
#minimize total resource consumption
costs = list()
for app in appList:
    cost = a2n_df.loc[app]*(rpa_df.loc[:, app]).item()
    #print("cost %s" %cost)
    costs.append(sum(cost))
total_cost = sum(costs)
print(total_cost)

0 +
0 +
a11*128 +
a12*128 +
a13*128 +
a14*128 +
a15*128 +
1280


In [43]:
opt = z3.Optimize()
constraints = a2n_domain + rsrc_constraint + mdd
#opt.add(constraints)
opt.assert_exprs(constraints)
p = 127
opt.minimize(latency_obj*p + (total_cost))
#opt.minimize(total_cost)
#opt.maximize(m)
opt.check()

opt_model = opt.model()
print("optimization model \n %s \n" %opt_model)

a2n_df_r = a2n_df.copy()
for app in nlappList:
    #print('App %s' %app)
    for node in nodeList:
        #print('Node %s' %node)
        a2n_df_r.loc[app, node] = opt_model.eval(a2n_df.loc[app, node])
#for app in appList:
#    for node in nodeList:
#        if a2n_df.loc[app,node] in opt_model:
#            print (a2n_df.loc[app,node])
        
print("app placement \n %s \n" %a2n_df_r)  

#Verify the results. 
print("latency cost: %s" %opt_model.eval(latency_obj*p))
print("resource cost: %s" %opt_model.eval(total_cost))

optimization model 
 [a11 = 0,
 a12 = 0,
 a13 = 0,
 a14 = 0,
 a15 = 1,
 z3name!1172 = 0,
 z3name!1173 = 0,
 z3name!1170 = 1,
 z3name!1171 = 0,
 z3name!1174 = 0] 

app placement 
      FSSN2 FSSN1 FSSN3 FSSN4 FSSN5
APP2     0     0     0     0     1
APP1     1     1     1     1     1 

latency cost: 508
resource cost: 1408


In [15]:
#import networx to color graph
# 5 nodes
# primary deployed everywhere
#weight optimization criteria
#visualize
#write up

In [16]:
#minimize latency equation
# App is on FSSN2 and Dependency is deployed once
m21 = a2n_df.loc['APP1','FSSN2'] \
    * (a2n_df.loc['APP2','FSSN1']) \
    * (1 - a2n_df.loc['APP2','FSSN2']) \
    * (1 - a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1']]).item()

m22 = a2n_df.loc['APP1','FSSN2'] \
    * (1 - a2n_df.loc['APP2','FSSN1']) \
    * (    a2n_df.loc['APP2','FSSN2']) \
    * (1 - a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN2']]).item()
        
m23 = a2n_df.loc['APP1','FSSN2'] \
    * (1 - a2n_df.loc['APP2','FSSN1']) \
    * (1 - a2n_df.loc['APP2','FSSN2']) \
    * (    a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN3']]).item()

# App is on FSSN2 and Dependency is deployed twice
m21_22 = a2n_df.loc['APP1','FSSN2'] \
    *     (a2n_df.loc['APP2','FSSN1']) \
    *     (a2n_df.loc['APP2','FSSN2']) \
    * (1 - a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1'], \
           n2n_df.loc['FSSN2','FSSN2']]).item()
    
m21_23 = a2n_df.loc['APP1','FSSN2'] \
    *     (a2n_df.loc['APP2','FSSN1']) \
    * (1 - a2n_df.loc['APP2','FSSN2']) \
    *     (a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1'], \
           n2n_df.loc['FSSN2','FSSN3']]).item()
    
m22_23 = a2n_df.loc['APP1','FSSN2'] \
    * (1 - a2n_df.loc['APP2','FSSN1']) \
    *     (a2n_df.loc['APP2','FSSN2']) \
    *     (a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN2'], \
           n2n_df.loc['FSSN2','FSSN3']]).item()

# App is on FSSN2 and Dependency is deployed three times    
m21_22_23 = a2n_df.loc['APP1','FSSN2'] \
    * (a2n_df.loc['APP2','FSSN1']) \
    * (a2n_df.loc['APP2','FSSN2']) \
    * (a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1'], \
           n2n_df.loc['FSSN2','FSSN2'], \
           n2n_df.loc['FSSN2','FSSN3']]).item()

m = m21+m22+m23+m21_22+m21_23+m22_23+m21_22_23
m

#a2n_df.loc[app,node] * a2n_df.loc[dep,node]
#a2a_df.loc[app] -> [1, 0]


1*a12*(1 - a11)*(1 - a13)*1 +
1*(1 - a12)*a11*(1 - a13)*0 +
1*(1 - a12)*(1 - a11)*a13*1 +
1*a12*a11*(1 - a13)*0 +
1*a12*(1 - a11)*a13*1 +
1*(1 - a12)*a11*a13*0 +
1*a12*a11*a13*0